In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from ipyleaflet import *
from ipywidgets import Layout
import os
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import pyodbc
from pyproj import Transformer

%matplotlib inline
%load_ext line_profiler

In [2]:
#find working folder
dir_path = os.getcwd()

In [3]:
def gen_spatial_data():
    sz = 13**3
    df = pd.DataFrame(np.random.randint(240000,380000, (sz, 1)), columns=['EASTING'])
    df['NORTHING'] = np.random.randint(35000, 130000, (sz, 1))
    df['date'] = pd.date_range('2000-01-01', freq='1S', periods=len(df))
    df['dt2'] = pd.date_range('1980-01-01', freq='999S', periods=len(df))
    df['f1'] = np.random.rand(len(df))
    df['f2'] = np.random.rand(len(df))
    return df

In [4]:
b = gpd.read_file(dir_path + "/data/geospatial/DSFRS_Service_Area.shp")
#b = b.to_crs("epsg:4326")
b = b.to_crs(epsg=4326)
b
#transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
#boundary = transformer.transform(boundary['geometry'])
#boundary.crs

,cat,ID,geometry
0,1,0,"POLYGON ((-3.51005 50.54224, -3.51042 50.54599..."


In [ ]:
# connection to server and databse
try:
    conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=SQL1\\LIVE1;"
                      "Database=CommonData;"
                      "Trusted_Connection=yes;",
                       timeout=1)
    # Read the sql file
    query = open(dir_path + '/' + 'ers_failures.sql', 'r')
    # read the sql query and connection and create a dataframe
    df = pd.read_sql(query.read(), conn)
    # convert x/y columns to number from text
    df[["EASTING", "NORTHING"]] = df[["EASTING", "NORTHING"]].apply(pd.to_numeric)
except:
    #print("Connetion to Database timed out")
    df = gen_spatial_data()
#print(df.info())

In [ ]:
#%timeit df

In [ ]:
def vectorized_convert(df):
    transformer = Transformer.from_crs("epsg:27700", "epsg:4326")
    converted = transformer.transform(df['EASTING'].tolist(), df['NORTHING'].tolist())
    df['lat'] = converted[1]
    df['lon'] = converted[0]
    return df
vec = vectorized_convert(df)
#%lprun -f vectorized_convert vectorized_convert(vec)

In [ ]:
coords = vec[['lat', 'lon']].values
miles = 2
conversion_factor = 0.62137119
kilometers = miles / conversion_factor
kms_per_radian = 6371.0088
epsilon = kilometers / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=12, metric='haversine', algorithm='ball_tree').fit(np.radians(coords))
cluster_labels = db.labels_
#print(cluster_labels)

In [ ]:
# calculate the number of clusters
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
print('Number of clusters: {}'.format(num_clusters))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print(clusters)
# identify core points
core_points = np.zeros_like(cluster_labels, dtype = bool)
core_points[db.core_sample_indices_] = True
#print(core_points)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(coords, cluster_labels))

In [ ]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)
#print(centermost_points)
#centermost_points.to_csv("centre.csv", index=False)
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lat':lats, 'lon':lons})
rs = rep_points
#rs = rep_points.apply(lambda row: df[(df['lat']==row['lat']) &amp;&amp; (df['lon']==row['lon'])].iloc[0], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=[18, 10])
rs_scatter = ax.scatter(rs['lat'], rs['lon'], c='#99cc99', edgecolor='None', alpha=0.7, s=300)
df_scatter = ax.scatter(df['lat'], df['lon'], c='k', alpha=0.9, s=3)
ax.set_title('Full data set vs DBSCAN reduced set')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')
plt.show()

In [ ]:
gdf = gpd.GeoDataFrame(rs, geometry=gpd.points_from_xy(rs['lat'], rs['lon']),crs="EPSG:4326")

geo_data = GeoData(geo_dataframe = gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

In [ ]:
import folium
import webbrowser
# Build map 
m = folium.Map(location=(50.909614, -3.48), zoom_start=9, zoomSnap=0.1, zoomDelta=0.5, tiles='cartodbpositron', control_scale=True)

# set custom name for base layer
#m = folium.Map((0, 0), tiles=None)
#folium.TileLayer('cartodbpositron', name='my tilelayer').add_to(m)

layer = folium.FeatureGroup(name='Clusters', show=True)
layer2 = folium.FeatureGroup(name='Points', show=True)


# add marker one by one on the map
for i in range(0,len(rs)):
   folium.CircleMarker(location=[rs.iloc[i]['lon'], rs.iloc[i]['lat']], radius=10, stroke=True, color='crimson', weight=2, fill=True,
      fill_color='crimson', fillOpacity=1).add_to(layer)

# add marker one by one on the map
for i in range(0,len(df)):
   folium.CircleMarker(location=[df.iloc[i]['lon'], df.iloc[i]['lat']], radius=1, stroke=True, color='green', weight=2, fill=True,
      fill_color='green', fillOpacity=1).add_to(layer2)

sw = df[['lon', 'lat']].min().values.tolist()
ne = df[['lon', 'lat']].max().values.tolist()

m.fit_bounds([sw, ne])
layer.add_to(m)
layer2.add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
#m.save('plot_data.html')
# Import the Folium interactive html file
#from IPython.display import HTML
#HTML('<iframe src=plot_data.html</iframe>')
#webbrowser.open('plot_data.html', new=2) 